---
layout: post
title: Trimester 2 Final Review (12 Weeks)
description: Review of 5 major things I did the last 12 weeks of Tri 2
comments: true
sticky_rank: 1
---

**1. Frontend Login Feature**

- implemented secure access to our server 
- Sent a request for a token, if not fufilled then access denied.
- The frontend sends a request (usually via HTTP POST) to the backend API with login details.
- For every request to a protected resource, the frontend includes the token in the Authorization header (Bearer Token).
The backend verifies the token before granting access.
- 25 Total Frontend Commits

- progress seen in scrum board: <img src="Screenshot 2025-03-03 at 12.07.18 AM.png" alt="Cantella Image" style="width:100%; max-width:600px;">


<img src="Screenshot 2025-03-02 at 11.51.27 PM.png" alt="Cantella Image" style="width:100%; max-width:600px;">

  **2. Chatbot API** 

  - Used Code to call Gemeni API in order to serve as a third party source to answer any questions
  - One of the first aditions to our website
  - Along the way added requests from postman to send and receieve requests in the backend
  - Started to work on messages saving in the backend instead of just locally
  - made requests save until deletion by user

<img src="Screenshot 2025-03-03 at 12.03.11 AM.png" alt="Cantella Image" style="width:100%; max-width:600px;">

<img src="{{site.baseurl}}/images/cantella_image.png" alt="Cantella Image" style="width:100%; max-width:600px;">

**3. Gemini ChatBot Frontend Integration**

- Handling User Input and Sending Questions to the Backend
- The sendQuestion(question) function is responsible for:

- Taking user input.
- Displaying it in the chatbox.
- Sending an API request to fetch a response from your Gemini chatbot.
- Dynamically displaying the chatbot’s response.

- sum up of all work seen in burndown list here: <img src="Screenshot 2025-03-03 at 12.20.30 AM.png" alt="Cantella Image" style="width:100%; max-width:600px;">

In [ ]:
async function sendQuestion(question) {
  const chatBox = document.getElementById("chat-box");

  // Display user's question
  chatBox.innerHTML += `
    <div data-q="${question}">
      <strong>You:</strong>
      <span class="question-text">${question}</span>
      <button onclick="deleteQ('${question}')">Delete</button>
      <button onclick="updateQ('${question}')">Update</button>
    </div>`;

  // Send request to backend
  const response = await fetch(`${pythonURI}/api/ai/help`, {
    ...fetchOptions,
    method: "POST",
    body: JSON.stringify({ question }),
  });

  // Get chatbot response and display it
  const data = await response.json();
  const aiResponse = data.response || "Error: Unable to fetch response.";

  chatBox.innerHTML += `
    <div data-r="${question}">
      <strong>CanTeach:</strong>
      <span class="response-text">${aiResponse}</span>
    </div>`;

  document.getElementById("question").value = ""; // Clear input field
  chatBox.scrollTop = chatBox.scrollHeight; // Auto-scroll
}


- Retreiving Past Conversaitons using command fetchAllLogs()
- Send a GET request to fetch all stored chatbot logs.
- Parse and process the response.
- Dynamically update the chat interface to display the retrieved conversation history.

In [ ]:
async function fetchAllLogs() {
  const chatBox = document.getElementById("chat-box");
  chatBox.innerHTML = ""; // Clear chat logs

  const response = await fetch(`${pythonURI}/api/ai/logs`, {
    ...fetchOptions,
    method: "GET",
  });

  const logs = await response.json();
  logs.forEach((log) => {
    const { question, response } = log;
    chatBox.innerHTML += `
      <div data-q="${question}">
        <strong>You:</strong>
        <span class="question-text">${question}</span>
        <button onclick="deleteQ('${question}')">Delete</button>
        <button onclick="updateQ('${question}')">Update</button>
      </div>
      <div data-r="${question}">
        <strong>CanTeach:</strong>
        <span class="response-text">${response}</span>
      </div>`;
  });
}


**4. ChatBot CRUD Options**

- After developing working backend and frontend integration involving "GET" aka (R), started working on Update and Delete
- Updating a question:
- Uses a PUT request to modify an existing user query and updates both the question and response dynamically.
- The function updateQ(oldQuestion) is triggered when the user clicks the "Update" button.
- A prompt window appears, asking the user to enter a new version of their question.
- If the user cancels or enters an empty string, the function exits early using return; (i.e., no update happens).


In [ ]:
async function updateQ(oldQuestion) {
  const newQuestion = prompt("Enter the updated question:");
  if (!newQuestion) return;

  const response = await fetch(`${pythonURI}/api/ai/update`, {
    ...fetchOptions,
    method: "PUT",
    body: JSON.stringify({ oldQuestion, newQuestion }),
  });

  const data = await response.json();
  if (data.error) {
    alert(`Error: ${data.error}`);
    return;
  }

  // Update the DOM with the new question and response
  const questionDiv = document.querySelector(`div[data-q="${oldQuestion}"]`);
  if (questionDiv) {
    questionDiv.querySelector(".question-text").innerText = newQuestion;
    questionDiv.setAttribute("data-q", newQuestion);
  }

  const responseDiv = document.querySelector(`div[data-r="${oldQuestion}"]`);
  if (responseDiv) {
    responseDiv.querySelector(".response-text").innerText = data.response;
    responseDiv.setAttribute("data-r", newQuestion);
  }
}


<img src="Screenshot 2025-03-03 at 12.30.01 AM.png" alt="Cantella Image" style="width:100%; max-width:600px;">

- Delete Option Crud
- The function deleteQ(question) is triggered when the "Delete" button is clicked.
- The frontend sends a DELETE request to /api/ai/delete with the question in the request body.
- The backend processes the request and removes the question-response pair from storage.
- made model file to store all this code, works on deployed website

In [ ]:
async function deleteQ(question) {
  const response = await fetch(`${pythonURI}/api/ai/delete`, {
    ...fetchOptions,
    method: "DELETE",
    body: JSON.stringify({ question }),
  });

  const questionDiv = document.querySelector(`div[data-q="${question}"]`);
  const responseDiv = questionDiv?.nextElementSibling; // Get next sibling (response)
  if (questionDiv) questionDiv.remove();
  if (responseDiv) responseDiv.remove();
}


**5. Crud Options In Backend**

- Retrieving Update, Retrieves JSON data (oldQuestion, newQuestion).
- Checks if both questions are provided; returns an error if missing.
- Finds the existing log for oldQuestion in the database.
- Generates a new response for newQuestion using the model.generate_content() method.
- Updates the log with the new question and response.
- Receive Data: The backend receives the PUT request with JSON data containing oldQuestion and newQuestion.
- Validation: The backend checks if both oldQuestion and newQuestion are provided. If not, it sends an error response (400 Bad Request).



In [ ]:
{
  "oldQuestion": "What is 2+2?",
  "newQuestion": "What is 3+3?"
}

{"error": "Both old and new questions are required."}


- Once the POST method was up and running, I wanted to implement the PUT method for updating questions. The idea was that if a user wanted to modify a previously asked question, they could do so and receive an updated response. Here's how I handled this:

- It receives both the oldQuestion and newQuestion from the frontend.
- It checks if the old question exists in the database.
- It generates a new response for the updated question.
- It updates the database with the new question and response.
- It returns the updated response to the frontend.
- This was a crucial step to allow users to modify questions and get fresh answers.

In [ ]:
@app.route('/api/ai/update', methods=['PUT'])
def update_ai_question():
    data = request.get_json()
    old_question = data.get("oldQuestion", "")
    new_question = data.get("newQuestion", "")
    if not old_question or not new_question:
        return jsonify({"error": "Both old and new questions are required."}), 400
    
    # Fetch the old log
    log = ChatLog.query.filter_by(_question=old_question).first()
    if not log:
        return jsonify({"error": "Old question not found."}), 404
    
    try:
        # Generate a new response for the new question
        response = model.generate_content(f"Your name is CanTeach. You are a homework help AI chatbot... Here is your prompt: {new_question}")
        new_response = response.text

        # Update the database entry
        log._question = new_question
        log._response = new_response
        db.session.commit()
        return jsonify({"response": new_response}), 200
    except Exception as e:
        print("Error during update:", e)
        return jsonify({"error": str(e)}), 500


- As the system grew, I realized it would be helpful to display the stored questions and responses to users. So, I added a GET method to fetch all the saved logs from the database and display them in the frontend. Here's how I implemented it:

- Fetches all entries from the ChatLog table in the database.
- Converts each log to a readable format and returns them as a JSON array.
- The frontend can then display these logs in a table or a chat interface

In [ ]:
@app.route('/api/ai/logs', methods=['GET'])
def fetch_all_logs():
    try:
        logs = ChatLog.query.all()
        return jsonify([log.read() for log in logs]), 200
    except Exception as e:
        print("Error fetching logs:", e)
        return jsonify({"error": str(e)}), 500


- Finally, I added the DELETE method to allow users to remove specific questions and responses. If a user wanted to delete a question they previously asked, they could simply click a button, and the backend would delete it from the database. Here’s the code for that:

- Takes the question to be deleted from the request body.
- Searches for that question in the database.
- If found, it deletes the log.
- If not found, it returns an error message.

In [ ]:
@app.route("/api/ai/delete", methods=["DELETE"])
def delete_ai_chat_logs():
    data = request.get_json()
    if not data:
        return jsonify({"error": "No data provided."}), 400
    log = ChatLog.query.filter_by(_question=data.get("question", "")).first()
    if not log:
        return jsonify({"error": "Chat log not found."}), 404
    log.delete()
    return jsonify({"response": "Chat log deleted"}), 200


**6. Role As Assistant Scrum-Master**

As the Assistant Scrum Master, I worked closely with Xavier to keep our team on track and ensure we were making steady progress throughout the trimester. We took a hands-on approach in guiding the team and making sure everyone was aligned with our goals.

I started by creating an issue (even though it was officially listed under Xavier's name) to clearly define everyone's roles and what we hoped to achieve by the end of the trimester. While Xavier officially created the issue, I was the one who took the time to break down each team member's responsibilities, so everyone knew exactly what was expected. Below the roles section, I added a brief overview outlining the key focus areas and goals for the team, helping us stay on the same page.

In addition to clarifying roles, Xavier and I worked closely on the Kanban board to make sure tasks were getting done efficiently. We kept a close eye on progress and made sure everyone was staying on track with their work. By creating burndown charts, we were able to monitor our progress and adjust things as needed to keep moving forward.

Through all of this, Xavier and I made sure to provide support and direction where needed, keeping communication clear and making sure everyone had what they needed to succeed. Together, we helped create an environment where everyone could contribute effectively, driving the team towards our shared goals.



- Burndown List / Roles
- https://github.com/users/XavierTho/projects/3?pane=issue&itemId=96847217&issue=XavierTho%7Ccantella_frontend%7C70 

<img src="Screenshot 2025-03-03 at 12.46.08 AM.png" alt="Cantella Image" style="width:100%; max-width:600px;">